Extra Credit Assignment :  Complete this Notebook to implement a DCGAN similar to what is described in the paper "Unsupervised Representation Learning with Deep Convolutional Generative Adversarial Networks" by Radford et al. (2016).https://arxiv.org/abs/1511.06434v2  

General guidelines
1. MAKE A COPY OF THIS NOTEBOOK TEMPLATE TO YOUR OWN DRIVE BEFORE REVISING. You can download your version of the notebook and submit it to CANVAS as a .ipynb notebook file
2. You will be using a celebrity faces dataset as input from kaggle and import code for this dataset is provided below
3. Do not exceed 8 epochs as this will increase the runtime beyond what is reasonable for extra credit


### Discriminator module

In [27]:
import sys
!{sys.executable} -m pip install tensorboardX

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3.9 -m pip install --upgrade pip


###  Imports

In [43]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.init as init
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.utils import save_image
from tensorboardX import SummaryWriter
from PIL import Image


In [29]:
class Discriminator(nn.Module):
    def __init__(self, channels_img, feature_d):
        super(Discriminator, self).__init__()
        self.disc = nn.Sequential(
            # Input: img_channels x 64 x 64
            nn.Conv2d(channels_img, feature_d, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2),
            #, inplace=True
            # Output: feature_d x 32 x 32
            nn.Conv2d(feature_d, feature_d * 2, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(feature_d * 2),
            nn.LeakyReLU(0.2),
            # Output: (feature_d*2) x 16 x 16
            nn.Conv2d(feature_d * 2, feature_d * 4, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(feature_d * 4),
            nn.LeakyReLU(0.2),
            # Output: (feature_d*4) x 8 x 8
            nn.Conv2d(feature_d * 4, feature_d * 8, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(feature_d * 8),
            nn.LeakyReLU(0.2),
            # Output: (feature_d*8) x 4 x 4
            nn.Conv2d(feature_d * 8, 1, kernel_size=4, stride=2, padding=0),
            
            # Output: 1 x 1 x 1
            nn.Sigmoid(),  
        )
        
    def forward(self, x):
        return self.disc(x)

### Generator Module

In [30]:
class Generator(nn.Module):
    def __init__(self, channels_noise, channels_img, features_g):
        super(Generator, self).__init__()
        self.gen = nn.Sequential(
            # Input: N x channels_noise x 1 x 1
            nn.ConvTranspose2d(channels_noise, features_g * 16, kernel_size=4, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(features_g * 16),
            nn.ReLU(),
            # Output: N x (features_g * 16) x 4 x 4
            nn.ConvTranspose2d(features_g * 16, features_g * 8, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(features_g * 8),
            nn.ReLU(),
            # Output: N x (features_g * 8) x 8 x 8
            nn.ConvTranspose2d(features_g * 8, features_g * 4, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(features_g * 4),
            nn.ReLU(),
            # Output: N x (features_g * 4) x 16 x 16
            nn.ConvTranspose2d(features_g * 4, features_g * 2, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(features_g * 2),
            nn.ReLU(),
            # Output: N x (features_g * 2) x 32 x 32
            nn.ConvTranspose2d(features_g * 2, channels_img, kernel_size=4, stride=2, padding=1),
            # Output: N x channels_img x 64 x 64
            nn.Tanh(),   #[-1,1]
        )

    def forward(self, x):
        return self.gen(x)

In [31]:
def initialize_weights(model):
        for m in model.modules():
            if isinstance(m, (nn.Conv2d, nn.ConvTranspose2d, nn.BatchNorm2d)):
                init.normal_(m.weight.data, 0.0, 0.02)
def test():
    N, channels_img, H, W = 8, 3, 64, 64
    channels_noise = 100
    x = torch.randn((N, channels_img, H, W))
    discriminator = Discriminator(channels_img, 8)
    initialize_weights(discriminator)
    assert discriminator(x).shape == (N, 1, 1, 1)
    generator = Generator(channels_noise, channels_img, 8)
    initialize_weights(generator)
    noise = torch.randn((N, channels_noise, 1, 1))
    assert generator(noise).shape == (N, channels_img, H, W)
    print("Success")

test()

Success


### Hyperparameters

In [46]:
#remember to specify CPU instead of GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
LEARNING_RATE = 2e-4
batch_size = 128
IMAGE_SIZE = 64
channels_img = 3
channels_noise = 100
NUM_EPOCHS = 1 #Do not exceed 10 for your final submission
features_d = 64
features_g = 64

generator = Generator(channels_noise, channels_img, features_g).to(device)
discriminator = Discriminator(channels_img, features_d).to(device)

initialize_weights(generator)
initialize_weights(discriminator)

gen_optimizer = optim.Adam(generator.parameters(), lr=LEARNING_RATE, betas=(0.5, 0.999))
disc_optimizer = optim.Adam(discriminator.parameters(), lr=LEARNING_RATE, betas=(0.5, 0.999))

# real_label = 1
# fake_label = 0

fix_noise = torch.randn(32, channels_noise, 1, 1).to(device)

writer_real = SummaryWriter(f"logs/real")
writer_fake = SummaryWriter(f"logs/fake")
step = 0

generator.train()
discriminator.train()

Discriminator(
  (disc): Sequential(
    (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.2)
    (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): LeakyReLU(negative_slope=0.2)
    (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (6): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): LeakyReLU(negative_slope=0.2)
    (8): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (9): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): LeakyReLU(negative_slope=0.2)
    (11): Conv2d(512, 1, kernel_size=(4, 4), stride=(2, 2))
    (12): Sigmoid()
  )
)

### Data-Loading and Image Transforms

In [48]:
# from google.colab import drive
# drive.mount('/content/drive')

# Specify the path to the destination directory
# destination_path = "/content/drive/MyDrive/CSS581-ML-Aut24/InputDataSets/img_align_celeba/img_align_celeba"
destination_path = "img_align_celeba"
transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),   # Resize images to (64, 64)
    transforms.ToTensor(),            # Convert image to tensor with range [0, 1]
    transforms.Normalize(
        [0.5 for _ in range(channels_img)], [0.5 for _ in range(channels_img)]),
])

# Update the root path to the extracted directory
dataset = datasets.ImageFolder(root=destination_path, transform=transform)

dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

### Define Losses

In [49]:
criterion = nn.BCELoss()

### Define Training Process

#### Seed the randomizer

In [ ]:
print("Starting training...")
for epoch in range(NUM_EPOCHS):
        for batch_idx, (data, _) in enumerate(dataloader):
          data = data.to(device)
          noise = torch.randn(batch_size, channels_noise, 1, 1).to(device)
          fake = generator(noise)
          
          ###Train discriminator: max log(D(x)) + log(1-D(G(z)))
          disc_real = discriminator(data).reshape(-1)
          lossD_real = criterion(disc_real, torch.ones_like(disc_real))

          disc_fake = discriminator(fake).reshape(-1)
          lossD_fake = criterion(disc_fake, torch.zeros_like(disc_fake)) 

          lossD = (lossD_real + lossD_fake) 
          discriminator.zero_grad()
          lossD.backward(retain_graph=True)
          disc_optimizer.step()

          ###Train generator: max log(D(G(z)))
          output = discriminator(fake).reshape(-1)
          lossG = criterion(output, torch.ones_like(output))
          generator.zero_grad()
          lossG.backward()
          gen_optimizer.step()

          if batch_idx % 100 == 0:
            print(f"Epoch [{epoch}/{NUM_EPOCHS}] Batch {batch_idx}/{len(dataloader)} \
            Loss D: {lossD:.4f}, Loss G: {lossG:.4f}")
            with torch.no_grad():
                fake = generator(fix_noise)
                img_grid_real = torchvision.utils.make_grid(data[:32], normalize=True)
                img_grid_fake = torchvision.utils.make_grid(fake[:32], normalize=True)
                writer_real.add_image("Real", img_grid_real, global_step=step)
                writer_fake.add_image("Fake", img_grid_fake, global_step = step)
            step += 1

Starting training...
Epoch [0/1] Batch 0/1583             Loss D: 1.3879, Loss G: 0.8136


### Check Results

In [44]:
def display_image(epoch_no):
     filename = f'generated_images_epoch_{epoch_no}.png'
     try:
        # Load the image
        img = Image.open(filename)

        # Display the image using matplotlib
        plt.figure(figsize=(8, 8))
        plt.imshow(transforms.ToPILImage()(transforms.ToTensor()(img)))
        plt.axis('off')  # Turn off axis for better visualization
        plt.title(f'Generated Images - Epoch {epoch_no}')
        plt.show()
     except FileNotFoundError:
        print(f"No image found for epoch {epoch_no}. Ensure the file '{filename}' exists.")

In [45]:
display_image(NUM_EPOCHS)

No image found for epoch 5. Ensure the file 'generated_images_epoch_5.png' exists.
